# CNN on mnist data
 

In [1]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('mnist_train.csv')
print(df.shape)

(60000, 785)


In [3]:
y = df['label'].values
X = df.drop(['label'],1).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [4]:
print(y_test.shape)

(9000,)


In [5]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)


In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)


MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


We have 784\*(250+1) + 250\*(100+1) + 100\*(10+1) = 222 360 parameters to train

In [9]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

In [10]:
fit(mlp, train_loader)

Epoch : 0 [0/51000 (0%)]	Loss: 7.191383	 Accuracy:15.625%
Epoch : 0 [1600/51000 (3%)]	Loss: 0.602311	 Accuracy:69.363%
Epoch : 0 [3200/51000 (6%)]	Loss: 0.100533	 Accuracy:77.104%
Epoch : 0 [4800/51000 (9%)]	Loss: 0.434235	 Accuracy:81.002%
Epoch : 0 [6400/51000 (13%)]	Loss: 0.410084	 Accuracy:83.085%
Epoch : 0 [8000/51000 (16%)]	Loss: 0.328846	 Accuracy:84.250%
Epoch : 0 [9600/51000 (19%)]	Loss: 0.244209	 Accuracy:85.185%
Epoch : 0 [11200/51000 (22%)]	Loss: 0.168041	 Accuracy:85.942%
Epoch : 0 [12800/51000 (25%)]	Loss: 0.482129	 Accuracy:86.518%
Epoch : 0 [14400/51000 (28%)]	Loss: 0.165036	 Accuracy:87.070%
Epoch : 0 [16000/51000 (31%)]	Loss: 0.051867	 Accuracy:87.681%
Epoch : 0 [17600/51000 (35%)]	Loss: 0.134861	 Accuracy:88.005%
Epoch : 0 [19200/51000 (38%)]	Loss: 0.220978	 Accuracy:88.384%
Epoch : 0 [20800/51000 (41%)]	Loss: 0.359432	 Accuracy:88.642%
Epoch : 0 [22400/51000 (44%)]	Loss: 0.112294	 Accuracy:88.949%
Epoch : 0 [24000/51000 (47%)]	Loss: 0.116700	 Accuracy:89.148%
Epoch 

Epoch : 4 [8000/51000 (16%)]	Loss: 0.003297	 Accuracy:97.087%
Epoch : 4 [9600/51000 (19%)]	Loss: 0.063884	 Accuracy:97.041%
Epoch : 4 [11200/51000 (22%)]	Loss: 0.103690	 Accuracy:97.053%
Epoch : 4 [12800/51000 (25%)]	Loss: 0.003757	 Accuracy:97.054%
Epoch : 4 [14400/51000 (28%)]	Loss: 0.063418	 Accuracy:96.882%
Epoch : 4 [16000/51000 (31%)]	Loss: 0.163261	 Accuracy:96.838%
Epoch : 4 [17600/51000 (35%)]	Loss: 0.087939	 Accuracy:96.835%
Epoch : 4 [19200/51000 (38%)]	Loss: 0.164431	 Accuracy:96.875%
Epoch : 4 [20800/51000 (41%)]	Loss: 0.082651	 Accuracy:96.913%
Epoch : 4 [22400/51000 (44%)]	Loss: 0.040517	 Accuracy:96.964%
Epoch : 4 [24000/51000 (47%)]	Loss: 0.093792	 Accuracy:96.925%
Epoch : 4 [25600/51000 (50%)]	Loss: 0.107988	 Accuracy:96.918%
Epoch : 4 [27200/51000 (53%)]	Loss: 0.004354	 Accuracy:96.937%
Epoch : 4 [28800/51000 (56%)]	Loss: 0.258705	 Accuracy:96.934%
Epoch : 4 [30400/51000 (60%)]	Loss: 0.013762	 Accuracy:96.964%
Epoch : 4 [32000/51000 (63%)]	Loss: 0.157610	 Accuracy:96

## MLP Evaluation

In [11]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))
evaluate(mlp)

Test accuracy:0.957% 


<center><h2>Convolutional Neural Network</h2></center>

## Data loader

Since a CNN needs a image shape as input let's reshape our flatten images to real image

In [12]:
torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)


torch.Size([51000, 1, 28, 28])
torch.Size([9000, 1, 28, 28])


In [13]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [14]:
fit(cnn,train_loader)

Epoch : 0 [0/51000 (0%)]	Loss: 14.271686	 Accuracy:18.750%
Epoch : 0 [1600/51000 (3%)]	Loss: 1.520592	 Accuracy:20.772%
Epoch : 0 [3200/51000 (6%)]	Loss: 1.020106	 Accuracy:38.150%
Epoch : 0 [4800/51000 (9%)]	Loss: 1.206845	 Accuracy:48.758%
Epoch : 0 [6400/51000 (13%)]	Loss: 0.656476	 Accuracy:55.706%
Epoch : 0 [8000/51000 (16%)]	Loss: 0.365748	 Accuracy:60.857%
Epoch : 0 [9600/51000 (19%)]	Loss: 0.429424	 Accuracy:64.327%
Epoch : 0 [11200/51000 (22%)]	Loss: 0.286801	 Accuracy:67.112%
Epoch : 0 [12800/51000 (25%)]	Loss: 0.710829	 Accuracy:69.288%
Epoch : 0 [14400/51000 (28%)]	Loss: 0.381004	 Accuracy:70.877%
Epoch : 0 [16000/51000 (31%)]	Loss: 0.533938	 Accuracy:72.443%
Epoch : 0 [17600/51000 (35%)]	Loss: 0.427061	 Accuracy:73.769%
Epoch : 0 [19200/51000 (38%)]	Loss: 0.329710	 Accuracy:75.005%
Epoch : 0 [20800/51000 (41%)]	Loss: 0.166885	 Accuracy:76.123%
Epoch : 0 [22400/51000 (44%)]	Loss: 0.526415	 Accuracy:77.046%
Epoch : 0 [24000/51000 (47%)]	Loss: 0.174274	 Accuracy:77.917%
Epoch

Epoch : 4 [6400/51000 (13%)]	Loss: 0.393897	 Accuracy:95.507%
Epoch : 4 [8000/51000 (16%)]	Loss: 0.050384	 Accuracy:95.630%
Epoch : 4 [9600/51000 (19%)]	Loss: 0.060093	 Accuracy:95.505%
Epoch : 4 [11200/51000 (22%)]	Loss: 0.065229	 Accuracy:95.450%
Epoch : 4 [12800/51000 (25%)]	Loss: 0.203727	 Accuracy:95.332%
Epoch : 4 [14400/51000 (28%)]	Loss: 0.112233	 Accuracy:95.288%
Epoch : 4 [16000/51000 (31%)]	Loss: 0.256157	 Accuracy:95.328%
Epoch : 4 [17600/51000 (35%)]	Loss: 0.057604	 Accuracy:95.338%
Epoch : 4 [19200/51000 (38%)]	Loss: 0.093119	 Accuracy:95.440%
Epoch : 4 [20800/51000 (41%)]	Loss: 0.290356	 Accuracy:95.445%
Epoch : 4 [22400/51000 (44%)]	Loss: 0.119448	 Accuracy:95.466%
Epoch : 4 [24000/51000 (47%)]	Loss: 0.182175	 Accuracy:95.460%
Epoch : 4 [25600/51000 (50%)]	Loss: 0.117912	 Accuracy:95.455%
Epoch : 4 [27200/51000 (53%)]	Loss: 0.150832	 Accuracy:95.450%
Epoch : 4 [28800/51000 (56%)]	Loss: 0.156462	 Accuracy:95.425%
Epoch : 4 [30400/51000 (60%)]	Loss: 0.698852	 Accuracy:95.

In [15]:
evaluate(cnn)

Test accuracy:0.953% 
